# Extract actual optimisation parameters to a table

Plot solution vectors from multiple MFILEs as a LateX table.

In [37]:
# Reload Process each time (keep editable install up-to-date)
%load_ext autoreload
%autoreload 2

import process.io.plot_solutions as ps
from pathlib import Path
import pandas as pd
from typing import Sequence

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
def opt_param_table(runs_metadata: Sequence[ps.RunMetadata]) -> pd.DataFrame:
    """Create an easy to read table of optimisation parameters for different runs.

    :param runs_metadata: metadata for mfiles to tabulate
    :type runs_metadata: Sequence[ps.RunMetadata]
    :return: dataframe in required form to plot
    :rtype: pd.DataFrame
    """
    # Create dataframe from runs metadata: mfile data with a tag for each run
    results_df = ps._create_df_from_run_metadata(runs_metadata)

    # Filter for tag, actual values of optimisation parameters and objective function
    opt_params_df = ps._filter_vars_of_interest(
        results_df, opt_param_value_pattern=ps.OPT_PARAM_VALUE_REGEX
    )

    # Now separate optimisation parameter values from their names
    opt_params_values_df = opt_params_df.filter(
        regex=f"{ps.OPT_PARAM_VALUE_REGEX}|{ps.TAG}"
    )
    opt_params_names_df = opt_params_df.filter(regex=ps.NORM_OPT_PARAM_NAME_REGEX)

    # Replace xcm--- optimisation parameter column headers with actual var names
    # in sub-df: allows plotting showing actual var names
    # eg. column headers "tag, xcm001, xcm002, ..." become
    # "tag, bt, rmajor, ..."
    # Normalising row may have been filtered out; reset index to ensure
    # opt param names in row 0
    opt_params_names_df_reset = opt_params_names_df.reset_index(drop=True)
    opt_params_names = opt_params_names_df_reset.loc[0].values.tolist()

    # Need to include tag column as first column header
    opt_params_names_with_tag = opt_params_names.copy()
    opt_params_names_with_tag.insert(0, ps.TAG)
    opt_params_values_with_names_df = opt_params_values_df.set_axis(
        opt_params_names_with_tag, axis=1
    )

    # This makes a wide df: difficult to present
    # Melt to simplify
    melted = opt_params_values_with_names_df.melt(id_vars="tag")
    # Pivot to get desired form
    pivoted = melted.pivot(index="variable", columns="tag", values="value")
    # Remove index name
    pivoted = pivoted.reset_index()
    return pivoted

In [39]:
data_dir = Path.cwd()
runs_metadata = [
    ps.RunMetadata(data_dir / "lt_MFILE.DAT", "min rmajor"),
    ps.RunMetadata(data_dir / "lt_max_q_MFILE.DAT", "max q"),
]

opt_param_table_df = opt_param_table(
    runs_metadata=runs_metadata,
)

opt_param_table_df = opt_param_table_df.rename(
    columns={
        "variable": "Parameter",
        "max q": "$\max Q$",
        "min rmajor": "$\min R_{major}$",
    }
)

# Add in latex and description cols
opt_param_desc = {
    "beta": [r"$\beta$", "Plasma beta"],
    "bore": ["$R_{CS}$", "Central solenoid inboard radius (m)"],
    "bt": ["$B_T$", "Toroidal field on axis (T)"],
    "coheof": [
        "$J_{CS}$",
        "Central solenoid overall current density at end of flat-top (A/m2)",
    ],
    "cpttf": ["$I_{TF, turn}$", "TF coil current per turn (A)"],
    "dene": ["$n_e$", "Electron density (/m3)"],
    "dr_tf_wp": ["$\Delta R_{WP}$", "Radial thickness of winding pack (m)"],
    "fcohbop": [
        "$f_{J_{CS}}$",
        "Ratio of central solenoid overall current density at beginning of pulse / end of flat-top",
    ],
    "fcutfsu": ["$f_{TF_{Cu}}$", "Copper fraction of cable conductor (TF coils)"],
    "fimp(13)": ["$f_{Xe}$", "Xenon density fraction relative to electron density"],
    "fvsbrnni": [
        "$f_{I_p, NI}$",
        "Fraction of the plasma current produced by non-inductive means",
    ],
    "hfact": ["$H$", "H factor on energy confinement times, radiation corrected"],
    "oh_steel_frac": ["$f_{CS_{steel}}$", "Central solenoid steel fraction"],
    "ohcth": ["$\Delta R_{CS}$", "Central solenoid thickness (m)"],
    "q": ["$q^{95}$", "Safety factor 'near' plasma edge, equal to q95"],
    "ralpne": [r"$f_{n_{\alpha}, n_e}$", "Thermal alpha density/electron density"],
    "rmajor": ["$R_{major}", "Plasma major radius (m)"],
    "tdmptf": [
        "$t_{TF, dis}$",
        "Fast discharge time for TF coil in event of quench (s)",
    ],
    "te": ["$T_e$", "Volume averaged electron temperature (keV)"],
    "tfcth": ["$\Delta R_{TF, in}$", "Inboard TF coil thickness (m)"],
    "thkcas": [
        "$\Delta R_{TF case, in}$",
        "Inboard TF coil case outer (non-plasma side) thickness (m)",
    ],
    "thwcndut": ["$\Delta R_{TF cond}$", "TF coil conduit case thickness (m)"],
}

# Add latex and descriptions for each param
desc_dict = {"Symbol": [], "Description": []}
for param in opt_param_table_df["Parameter"].values:
    desc_dict["Symbol"].append(opt_param_desc[param][0])
    desc_dict["Description"].append(opt_param_desc[param][1])

desc_df = pd.DataFrame(desc_dict)
opt_param_table_df = pd.concat((opt_param_table_df, desc_df), axis="columns")

# Re-arrange col order
cols = opt_param_table_df.columns.to_list()
cols = [cols[0]] + [cols[3]] + [cols[2]] + [cols[1]] + [cols[4]]
opt_param_table_df = opt_param_table_df[cols]

# Make latex table
opt_param_table_df.to_latex(
    "opt_params_table.tex",
    float_format="%.2e",
    index=False,
    longtable=True,
    column_format="llrrp{0.45\linewidth}",
)
opt_param_table_df

,Parameter,Symbol,$\min R_{major}$,$\max Q$,Description
0,beta,$\beta$,4.011916e-02,4.121113e-02,Plasma beta
1,bore,$R_{CS}$,2.047501e+00,2.379342e+00,Central solenoid inboard radius (m)
2,bt,$B_T$,5.026924e+00,5.446529e+00,Toroidal field on axis (T)
3,coheof,$J_{CS}$,1.952060e+07,1.646238e+07,Central solenoid overall current density at en...
4,cpttf,"$I_{TF, turn}$",7.843945e+04,8.285887e+04,TF coil current per turn (A)
5,dene,$n_e$,8.457173e+19,8.162611e+19,Electron density (/m3)
6,dr_tf_wp,$\Delta R_{WP}$,5.169788e-01,7.075876e-01,Radial thickness of winding pack (m)
7,fcutfsu,$f_{TF_{Cu}}$,8.669537e-01,7.459364e-01,Copper fraction of cable conductor (TF coils)
8,fimp(13),$f_{Xe}$,6.764877e-04,8.994541e-04,Xenon density fraction relative to electron de...
9,fvsbrnni,"$f_{I_p, NI}$",4.772641e-01,4.188966e-01,Fraction of the plasma current produced by non...


In [40]:
# Drop params col
opt_param_table_no_params_df = opt_param_table_df.drop(columns=["Parameter"])
opt_param_table_no_params_df.rename(columns={"Symbol": "Parameter"}, inplace=True)

# Only keep key differences
for index, row in opt_param_table_no_params_df.iterrows():
    a = row["$\min R_{major}$"]
    b = row["$\max Q$"]
    diff = abs((a - b) / ((a + b) / 2))
    if diff < 0.2:
        opt_param_table_no_params_df.drop(index, inplace=True)

# Make latex table
opt_param_table_no_params_df.to_latex(
    "key_opt_params_table.tex",
    float_format="%.2e",
    index=False,
    # longtable=True,
    column_format="llrrp{0.45\linewidth}",
)
opt_param_table_no_params_df

,Parameter,$\min R_{major}$,$\max Q$,Description
6,$\Delta R_{WP}$,0.516979,0.707588,Radial thickness of winding pack (m)
8,$f_{Xe}$,0.000676,0.000899,Xenon density fraction relative to electron de...
11,$f_{CS_{steel}}$,0.511524,0.709818,Central solenoid steel fraction
16,"$t_{TF, dis}$",20.235901,26.565316,Fast discharge time for TF coil in event of qu...
17,$T_e$,13.022663,15.921998,Volume averaged electron temperature (keV)
19,"$\Delta R_{TF case, in}$",0.217757,0.296794,Inboard TF coil case outer (non-plasma side) t...
20,$\Delta R_{TF cond}$,0.008140,0.011035,TF coil conduit case thickness (m)
